In [1]:
import open3d as o3d
import numpy as np
import copy as cp
import open3d.core as o3c
import matplotlib.pyplot as plt
import pyransac3d as pyrsc
from scipy.spatial.transform import Rotation
from iteration_utilities import deepflatten

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#load pcd file, filter, downsample
pcdn = o3d.io.read_point_cloud("loop.pcd")
pcdn.estimate_normals()
cl, ind = pcdn.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=0.8)

pcd = pcdn.select_by_index(ind)
pcd = pcd.voxel_down_sample(voxel_size=0.1)
pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(40)
o3d.visualization.draw_geometries([pcd])

In [3]:
def create_cuboid(plane_eq):
    # Extract the coefficients A, B, C, and D from the plane equations
    A1, B1, C1, D1 = plane_eq[0]
    A2, B2, C2, D2 = plane_eq[1]
    A3, B3, C3, D3 = plane_eq[2]

    # Calculate the intersection lines between the planes
    line1 = np.cross([A1, B1, C1], [A2, B2, C2])
    line2 = np.cross([A2, B2, C2], [A3, B3, C3])
    line3 = np.cross([A3, B3, C3], [A1, B1, C1])

    # Calculate the corner points of the cuboid
    corner_points = []
    for i in range(2):
        for j in range(2):
            for k in range(2):
                corner_point = (
                    line1 * i / np.linalg.norm(line1) +
                    line2 * j / np.linalg.norm(line2) +
                    line3 * k / np.linalg.norm(line3)
                )
                corner_points.append(corner_point)

    return corner_points

In [4]:
points = np.asarray(pcd.points) # Load your point cloud as a numpy array (N, 3)

cube1 = pyrsc.Cuboid()
best_eq, best_inliers = cube1.fit(points, thresh=0.1, maxIteration=5000)
box_points = create_cuboid(best_eq)

o3d_box_points = o3d.utility.Vector3dVector(box_points)


In [5]:
print(best_eq)
test = o3d.geometry.OrientedBoundingBox.create_from_points(o3d_box_points)
test.color = (1,0,0)
part_pcd = pcd.select_by_index(best_inliers)
part_pcd.paint_uniform_color((1,0,0))

o3d.visualization.draw_geometries([pcd, part_pcd, test])

[[-0.01775066  0.00841708  0.99980701  0.62060831]
 [-0.02535879 -0.99964668  0.0079655  -1.66161568]
 [ 0.99952081 -0.0252125   0.01795784  5.25616302]]


In [6]:
oboxes = pcd.detect_planar_patches(
normal_variance_threshold_deg=50,
coplanarity_deg=85,
outlier_ratio=0.75,
min_plane_edge_length=1,
min_num_points=10,
search_param=o3d.geometry.KDTreeSearchParamKNN(knn=50))

print("Detected {} patches".format(len(oboxes)))

geometries = []
for obox in oboxes:

    mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(obox, scale=[1, 1, 0.0001])
    mesh.paint_uniform_color(obox.color)
    mesh.compute_triangle_normals()
    mesh.compute_vertex_normals()
    normals = np.asarray(mesh.vertex_normals)

    
    geometries.append(mesh)
    geometries.append(obox)

#o3d.visualization.draw_geometries(geometries + [boundarys.to_legacy()])
o3d.visualization.draw_geometries([pcd] + geometries)

Detected 36 patches


In [15]:
def filter_by_normal(pcd):
    ind = []
    for i in range(len(pcd.points)):
        if ((np.abs(pcd.normals[i][2]) < 0.3) and ((np.abs(pcd.normals[i][0]) > 0.8) or (np.abs(pcd.normals[i][1]) > 0.8))):
            pcd.points[i][2] = np.random.rand()*0.1
            pass
        else:
            ind.append(i)
    result = pcd.select_by_index(ind, invert=True)
    return result

In [16]:
filtered_pcd = filter_by_normal(pcd)
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(filtered_pcd, 0.1)
o3d.visualization.draw_geometries([voxel_grid])

#print(np.asarray(filtered_pcd.points[0:10]))


In [9]:
def create_box_at_point(point):
    
    box = o3d.geometry.TriangleMesh.create_box(0.1,0.1,0.1)
    box.paint_uniform_color([0,1,0])
    
    box.translate(point, False)
    
    
    return box
    

In [10]:
def get_points_on_line(point1, point2, distance):
    # Calculate the direction vector of the line
    direction_vector = point2 - point1

    # Calculate the length of the line
    line_length = np.linalg.norm(direction_vector)

    # Calculate the number of points based on the given distance
    num_points = int(line_length / distance) + 1

    # Calculate the step size between each point
    if (num_points <= 1):
        num_points += 1
    step_size = 1.0 / (num_points - 1)

    # Calculate the parameter values for each point on the line
    parameters = np.arange(0, 1 + step_size, step_size)

    # Calculate the points on the line using the parameter values
    points = [point1 + parameter * direction_vector for parameter in parameters]

    return points

In [17]:
n_points = len(filtered_pcd.points)
line_points = [(np.random.randint(n_points), np.random.randint(n_points)) for i in range(0,n_points,10)]
lineset = o3d.geometry.LineSet.create_from_point_cloud_correspondences(filtered_pcd, filtered_pcd, line_points)

#o3d.visualization.draw_geometries([lineset])
boxes = []
indexes = []
intersection_points = []
int_points = o3d.geometry.PointCloud()
for i in range(len(line_points)):
    p1 = lineset.get_line_coordinate(i)[0]
    p2 = lineset.get_line_coordinate(i)[1]
  
    linepoints = get_points_on_line(p1,p2,0.1)

    lp_o3d = o3d.utility.Vector3dVector(linepoints)

    is_included = voxel_grid.check_if_included(lp_o3d)

    
    for bl,x in zip(is_included, range(len(is_included))):
        if bl:
            indexes.append(voxel_grid.get_voxel(lp_o3d[x]))

    intersection_points.append([linepoints[x] for x in range(len(linepoints))  if not  is_included[x]] )

intersection_points = list(deepflatten(intersection_points, depth=1))

int_points.points.extend(o3d.utility.Vector3dVector(intersection_points))  
   

o3d.visualization.draw_geometries([voxel_grid] + [int_points])

In [39]:
oboxes = filtered_pcd.detect_planar_patches(
normal_variance_threshold_deg=50,
coplanarity_deg=85,
outlier_ratio=0.75,
min_plane_edge_length=2,
min_num_points=10,
search_param=o3d.geometry.KDTreeSearchParamKNN(knn=50))

print("Detected {} patches".format(len(oboxes)))

geometries = []
for obox in oboxes:

    mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(obox, scale=[1, 1, 0.0001])
    mesh.paint_uniform_color(obox.color)
    mesh.compute_triangle_normals()
    mesh.compute_vertex_normals()
    normals = np.asarray(mesh.vertex_normals)

    
    geometries.append(mesh)
    geometries.append(obox)

#o3d.visualization.draw_geometries(geometries + [boundarys.to_legacy()])
o3d.visualization.draw_geometries(geometries+[int_points])

Detected 14 patches
